# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [116]:
import pandas as pd
import flatiron_stats as fs
import scipy.stats as stats
import numpy as np
df = pd.read_csv('homepage_actions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [138]:
control = df.loc[df['group'] == 'control']
experiment = df.loc[df['group'] == 'experiment']

In [150]:
control['count'] = control['action'].apply(lambda x: 1 if x == 'click' else 0)
experiment['count'] = experiment['action'].apply(lambda x: 1 if x == 'click' else 0)

<ipython-input-150-7edfcc8e95ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['count'] = control['action'].apply(lambda x: 1 if x == 'click' else 0)
<ipython-input-150-7edfcc8e95ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment['count'] = experiment['action'].apply(lambda x: 1 if x == 'click' else 0)


In [160]:
pivot_c = control.pivot(index = ['id'], columns = 'action',values= 'count' )
pivot_c.fillna(0, inplace= True)
pivot_e = experiment.pivot(index = ['id'], columns = 'action',values= 'count' )
pivot_e.fillna(0, inplace= True)


<ipython-input-102-7edfcc8e95ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['count'] = control['action'].apply(lambda x: 1 if x == 'click' else 0)
<ipython-input-102-7edfcc8e95ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment['count'] = experiment['action'].apply(lambda x: 1 if x == 'click' else 0)


In [107]:
control.duplicated().value_counts()

False    4264
dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [161]:
p_val = fs.p_value_welch_ttest(pivot_c.click, pivot_e.click)
p_val

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [166]:
click_rate = np.sum(pivot_c.click)/len(pivot_c.click)
click_rate

0.2797118847539016

In [174]:
expected_clicks = click_rate * len(pivot_e)
expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [252]:
variance = click_rate * len(pivot_e) * (1 - click_rate)    
std = np.sqrt(variance)
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [254]:
z_score = (pivot_e.click.sum() - expected_clicks)/ (std)
z_score

3.6625360854823588

In [256]:
1 - stats.norm.cdf(z_score)

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.